In [1]:
import os

import tifffile as tf
import pandas as pd
import numpy as np

import scipy.ndimage as ndi

import skimage.measure as measure

import line_utils
import image_utils

In [2]:
# Establish the path the Excel file
base_path = "/Users/zachcm/Documents/Projects/ExM_Nadja/01_Macros_Analysis/"
workbook = "20241008_septin2_tubulin.xlsx"
workbook_path = os.path.join(base_path, workbook)
expected_colnames=['Distance_(microns)', 'MTs', 'septin2', 'DAPI']

In [3]:
# Load the table of contents
toc = pd.read_excel(workbook_path, sheet_name="ToC + P-t-p", header=2)

In [4]:
# load FWHM_along, which contains other statistics
metrics = pd.read_excel(workbook_path, sheet_name="FWHM_along", header=0).dropna(axis=0)

In [5]:
metrics = line_utils.merge_df_information(toc, 
                                          metrics, 
                                          id_key="Label", 
                                          mapped_keys=["dX (µm)", "dX (pxl)", "X1", "X2", "Angle","X","Y","length"])
metrics = metrics[~metrics['Y'].isna()]
# metrics = metrics.dropna()

In [6]:
# Now let's get the original images...
import glob

# ----- max proj files --- #
# max_proj_path = "/Volumes/Ries_Ewers/Septin2-GFP/MaxIPs"
# max_proj_files = glob.glob(max_proj_path+"/*.tif")

# # ...and associate the file with each metrics entry
# metrics["filename"] = ""
# for i, ml in metrics.iterrows():
#     for fn in max_proj_files:
#         if ml["Label"] in fn:
#             metrics.loc[i, "filename"] = fn
#             break

# ---- 3D files ---- #
image_directory = "/Volumes/Ries_Ewers/Septin2-GFP"
image_files = glob.glob(image_directory+"/*.nd")

for i, ml in metrics.iterrows():
    file_stub = os.path.splitext(ml["Label"])[0]
    for fn in image_files:
        if file_stub in fn:
            metrics.loc[i, "filename"] = fn
            break

In [7]:
groups = metrics.groupby('Unnamed: 0')

plot_stack = None
n_groups = len(groups)
n_ch = 3
length = 500
l2 = length // 2
group_img = np.zeros((n_groups, n_ch, length, length))
metrics['dX2'] = np.nan
metrics['X12'] = np.nan
metrics['X22'] = np.nan
for group, tup in enumerate(groups):
    name, entries = tup
    n_group = len(entries)
    # mean_dX = entries['dX (pxl)'].mean()
    im_proj = {}
    for i, ml in entries.iterrows():
        # Get the image associated with this row

        # ---- If max proj -----
        # im = tf.imread(ml["filename"])
        # ---- Elif mean proj ----
        im = image_utils.NDImage(ml["filename"])
        im = im[:].mean(1).squeeze()
        im_proj[i] = im
        # --- End If ----

        im = im/im.sum(-1).sum(-1)[:,None, None]


        # get x, y, angle for this row
        x, y, angle = ml[["X", "Y", "Angle"]]

        # Compute the line end points
        xl, xu, yl, yu = line_utils.get_line_profile_endpoints(x, y, angle, length)

        chs = measure.profile_line(im.T, 
                                [xl, yu], 
                                [xu, yl], 
                                linewidth=25)

        mt, septin, dapi = chs.T

        # --------- Fit septin peaks ---------

        # Find peaks in the septin channel
        peaks, peak_props, septin_threshold = line_utils.get_peaks(septin)
        # weight the peaks by the MT channel signal
        w, profile_center, sig = line_utils.compute_peak_weights(mt, peaks)
        # Find the two peaks most likely to be septin rings
        p0, p1 = line_utils.find_two_best_peaks(peaks, peak_props, septin_threshold, profile_center, w)
        
        # Get the distance between the peaks
        dX2 = np.abs(p0-p1)

        if (ml["Unnamed: 0"] != "A") and (ml["Unnamed: 0"] != "BA"):
            metrics.loc[i,['X12','X22','dX2']] = [p0, p1, dX2]


    mean_dX2 = entries['dX2'].mean()

    for i, ml in entries.iterrows():
        # Get the image associated with this row
        # ---- If max proj -----
        # im = tf.imread(ml["filename"])
        # ---- Elif mean proj ----
        im = im_proj[i]
        # --- End If ----
        im = im/im.sum(-1).sum(-1)[:,None, None]

        # get x, y, angle for this row
        x, y, angle = ml[["X", "Y", "Angle"]]

        # Rotate the image  # CYX
        im_rot = image_utils.pad_rot_and_trans_im(im, angle, x, y)

        # Crop the image
        xc, yc = im_rot.shape[2]//2, im_rot.shape[1]//2
        im_crop = im_rot[:,(yc-length):(yc+length),(xc-length):(xc+length)]

        # rescale the image
        # if np.isnan(ml["dX (pxl)"]):
        if np.isnan(ml["dX2"]):
            im_zoom = im_crop
        else:
            # mag = ml["dX (pxl)"]/mean_dX
            mag = ml["dX2"]/mean_dX2
            im_zoom = ndi.zoom(im_crop, (1,1,mag))

        # crop the image 
        xc, yc = im_zoom.shape[2]//2, im_crop.shape[1]//2
        im_crop2 = im_zoom[:,(yc-l2):(yc+l2),(xc-l2):(xc+l2)]

        # Get the rescaling to the septin endpoints
        # xx_norm = line_utils.rescale_inds(length, int(ml["X1"]), int(ml["X2"]))
        # xx_norm = mean_dX(2*xx_norm - 1)  # shift and scale

        # crop it again
        # if im_zoom.shape[2] > im_crop.shape[2]:
        #     xc2, yc2 = im_zoom.shape[2]//2, im_zoom.shape[1]//2
        #     im_crop2 = im_zoom[:,(yc2-l2):(yc2+l2),(xc2-l2):(xc2+l2)]
        # elif im_zoom.shape[2] < im_crop.shape[2]:
        #     im_crop2 = np.zeros((n_ch, length, length))
        #     d2 = (length - im_zoom.shape[2])/2
        #     if d2 == int(d2):
        #         d2 = int(d2)
        #         im_crop2[...,d2:-d2] = im_zoom
        #     else:
        #         d2 = int(d2)
        #         im_crop2[...,d2:(-d2-1)] = im_zoom
        group_img[group,...] += (im_crop2/n_group)
    

In [31]:
group_order = list(groups['Unnamed: 0'].unique().keys())
true_order = ["RC", "CS", "RS", "SM", "BA", "A"]
group_img_sorted = [group_order.index(g) for g in true_order]
print(group_img_sorted)

[3, 2, 4, 5, 1, 0]


In [11]:
tf.imwrite('pseudotime_images.ome.tif', group_img[group_img_sorted,...], metadata={'axes': 'TCYX'}, dtype=group_img.dtype)